In [ ]:
import os
import sys

import pandas as pd
import numpy as np
import datetime

from django.db import IntegrityError

from tv import models as tv_models


# Change DataFrame dtypes to match Database models
def model_for_db(csv):
    """
    @parameters:
    - csv: Electronic Guide filename in comma separated values (.csv)

    @returns:
    - Dataframe with items formatted for bmat tv/av programs
    """

    # Load csv into Dataframe
    df = pd.read_csv(csv)

    # Fill n/a with readable string "n/a"
    df = df.fillna('n/a')
    df['program_year'] = df['program_year'].astype(str)
    df['start_time'] = (df['start_date'].astype(str) + ' ' + df['start_time']).map(
        lambda x: datetime.datetime.strptime(x, '%Y%m%d %H:%M')
    )
    df['duration_in_seconds'] = pd.to_timedelta(df['duration_in_seconds'], unit='seconds')
    df['end_time'] = df['start_time'] + df['duration_in_seconds']
    df = df.drop(['start_date', 'duration_in_seconds'], axis=1)
    return df


# Get or create Program with channel obj and row data
def create_program(row, channel):
    if row.program_year != 'n/a':
        year = row.program_year[0:4]
    else:
        year = row.program_year
    program, created =tv_models.Program.objects.get_or_create(uid=row.program_id, channel=channel,
                                                            year=year,
                                                            local_title=row.program_local_title,
                                                            original_title=row.program_original_title
                                                            )
    if created:
        program.save()
        print("Program {} with id {} added to DB.".format(program.local_title,
                                                          program.uid,
                                                          ))
    return program


def create_channel(row):
    channel, created = tv_models.Channel.objects.get_or_create(uid=row.channel_id,
                                                     name=row.channel_name,
                                                     country_code=row.channel_country
                                                     )
    if created:
        channel.save()
        print('Channel {} from {} with id {} added to DB.'.format(channel.name,
                                                                  channel.country_code,
                                                                  channel.id
                                                                  ))
    return channel


def create_showtime(row, program):
    showtime, created = tv_models.Showtime.objects.get_or_create(program=program,
                                                              start_time=row.start_time,
                                                              end_time=row.end_time
                                                              )
    if created:
        showtime.save()
        print('Showtime starting on {} added to '.format(row.start_time)+program.local_title)
    return showtime


# Create Showtimes from DataFrame
def add_showtimes_from_dataframe(df, filename):
    for index, row in df.iterrows():
        channel = create_channel(row)
        program = create_program(row, channel)
        create_showtime(row, program)

    print("Finished adding programs from {}".format(filename))


def main():
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    filename = None
    while filename is None:
        filename = input('Please enter the full path to the Electronic Program Guide: ')
        if os.path.isfile(filename):
            os.chdir(filename[0:-len(filename.split('/')[-1])])
            print ("Uploading programs to database...")
            df = model_for_db(filename.split('/')[-1])
            add_showtimes_from_dataframe(df, filename)
            print ("Finished uploading programs from {} to database.".format(filename))
            return
        else:
            filename = None
            print ("File does not exist. Check the path is correct and try again.")


In [ ]:
! ls .
'../../data/epg.csv'

In [ ]:
df = model_for_db('../../data/epg.csv')

In [ ]:
add_showtimes_from_dataframe(df, 'epg.csv')

In [ ]:
df

# Vericast tool

In [2]:
import calendar
import datetime
import dateutil.parser

def utc_to_local(utc_dt):
    # get integer timestamp to avoid precision lost
    timestamp = calendar.timegm(utc_dt.timetuple())
    local_dt = datetime.datetime.fromtimestamp(timestamp)
    assert utc_dt.resolution >= datetime.timedelta(microseconds=1)
    return local_dt.replace(microsecond=utc_dt.microsecond)


In [5]:
import os
import json
import pandas as pd


            

# Change directory to filesdir
os.chdir('../../data/')
class VericastMatchReporter:
    """
    Handles ISO 8601 format datetimes.
    @params: filename, start_date, start_time, end_date, end_time, time_zone
    """
    
    df = pd.DataFrame(columns=['title','length','album','artist','start_time_utc'])
    matches_between_dates = None
    start_time = None
    end_time = None
    report_name = 'vericast-api-matches'
    report_file_extension = 'xslx'
    
    def __init__(self, **kwargs):
        """Initilize report, make a pandas dataframe with the matches."""
        self.filename = kwargs['filename']
        self.channel = self.filename.split('/')[-1].split('-')[-1].split('.')[0]
        self.report_name += '-'+self.channel 
        try:
            self.start_time = dateutil.parser.parse(kwargs['start_date']+'T'+kwargs['start_time']+kwargs['tz_offset'])
            self.end_time = dateutil.parser.parse(kwargs['end_date']+'T'+kwargs['end_time']+kwargs['tz_offset'])
        except:
            raise
        with open(self.filename) as f:
            for line in f.readlines():
                # Append match into df
                match = json.loads(line)
                self.df = self.df.append(match, ignore_index=True)
                
        # Cast dates as aware datetime 
        self.df['start_time_utc'] = pd.to_datetime(self.df['start_time_utc'], yearfirst=True, utc=True)
        print('Reporter initialized.')
        
    def create_report_between_times_xlsx(self):
        # Localize UTC for datetime filtering
        self.df['localized_time'] = self.df['start_time_utc'].map(lambda x: utc_to_local(x))
        self.df = self.df.set_index('localized_time')
        
        # matches = self.df.loc[(self.df['start_time_utc'] > self.start_time) & (self.df['start_time_utc'] < self.end_time)]
        matches = self.df.between_time(self.start_time, self.end_time)
        return matches
        
        return 'File {} has been created'.format()
    
    def __str__(self):
        return 'Vericast API matches from '+self.channel.replace('_', ' ').upper()+' between times '+str(self.start_time)+' and '+str(self.end_time)+' UTC.'

    def __repr__(self):
        return "<{}: Channel {}>".format(self.__class__.__name__, self.channel)


In [6]:
report = VericastMatchReporter(
    filename='matches-la_sexta.json', 
    start_date='2018-01-29', 
    start_time='12:00:00', 
    end_date='2018-02-01',
    end_time='00:00:00',
    tz_offset='+0500'
)

Reporter initialized.


In [7]:
print(type(report.start_time))
print(report.df.index)
report.create_report_between_times_xlsx()

<type 'datetime.datetime'>
RangeIndex(start=0, stop=474, step=1)


AssertionError: 

In [ ]:
df['start_time_utc']).map(
        lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M')
    )

In [ ]:
report_df = df.where((df.start_time_utc >= date) & (df.start_time_utc <= date2))

In [ ]:

df.start_time_utc.between_time()

In [ ]:
i = pd.date_range('2018-04-09', periods=4, freq='1D20min')

ts = pd.DataFrame({'A': [1, 2, 3, 4]}, index=i)

ts

In [ ]:
ts.between_time('0:15', '0:45')

In [ ]:
i

In [ ]:
start_time = '2018-02-15 21:22:12+00:00'
#end_time = datetime.datetime.strptime('2018-02-16 07:28:59+00:00', '')
end_time = '2018-02-16 07:28:59+00:00'
df['start_time_utc']   #.astype(str).between_time(start_time, end_time)


In [ ]:
df.head(1)